# Visualizing spatio-chemical filters with pythreejs
## Installation:
For running this notebook, the following additional packages are required:

pythreejs (https://pythreejs.readthedocs.io/en/stable/)

ipywebrtc==0.5.0

ipywidgets==7.5.1  

Installation with pip should work with a recent jupyter version. Reboot the jupyter notebook afterwards.



## Important: Cells must be executed one at time (rather than with Cell -> Run All)



# Calculate filter specificities (precomputed)
The first step is to calculate the filter specificities.
For the atomic filters, we must simply extract the neighborhood embedding layer parameters from the keras model object.
For the amino acid filters, since the amino-acid specificity of individual gaussian components is *non-linear*, we must first determine it on a set of test proteins. This is done as follows:
1. Calculate, for each gaussian component of each filter, the distribution of activities.
2. Identify the top-1/5\% activating residues.
3. Determine their amino acid type, secondary structure and accessible surface area.
4. Determine the mean activity across the top-activating residues.

For the trained networks, the specificities are precomputed. Otherwise, calculating the amino acid filter specificities takes about 1-2 hour on a laptop
To avoid this step and visualize only the atom filters, specify only_atom=True

Then, for a given filter, display the gaussians with highest mean activity (threshold1 = 33\% of the maximum mean activity), and show attribute specificity as inset.



In [2]:
from visualizations import show_3d_filters,weight_logo_3d
from utilities import dataset_utils
import numpy as np

mode = 'interface' # Prediction mode: 'interface' (protein-protein binding sites), 'epitope' (B-cell epitopes), 'idp' (protein - intrinsically disordered proteins binding sites)
use_MSA = False # Whether to use evolutionary information or not.



if mode == 'interface':
    top_percent = 5
    if use_MSA:
        model_name = 'ScanNet_PPI'
    else:
        model_name = 'ScanNet_PPI_noMSA'
    dataset_name = 'PPBS_validation'
    list_origins = np.concatenate([dataset_utils.read_labels('datasets/PPBS/labels_%s.txt'%dataset)[0]
     for dataset in ['validation_70','validation_homology','validation_topology','validation_none']
        ])
    
elif mode == 'epitope':
    top_percent = 5
    if use_MSA:
        model_name = 'ScanNet_PAI_0'
    else:
        model_name = 'ScanNet_PAI_noMSA_0'
    dataset_name = 'BCE_fold1'
    list_origins = dataset_utils.read_labels('datasets/BCE/labels_fold1.txt')[0]
    
elif mode == 'idp':
    top_percent = 5
    if use_MSA:
        model_name = 'ScanNet_PIDPI_0'
    else:
        model_name = 'ScanNet_PIDPI_noMSA_0'
    dataset_name = 'PIDPBS_fold0'
    list_origins = None
#     list_origins = dataset_utils.read_labels('datasets/PIDP/labels_fold0.txt')[0]
        
    


filter_specificities = show_3d_filters.calculate_filter_specificities(
    model_name,
    dataset_name = dataset_name,
    dataset_origins = list_origins,
    biounit=False,
    ncores=4,
    only_atom=False,
    top_percent = top_percent,
    fresh = False,
    Lmax = 1024

)




/Users/jerometubiana/Documents/GitHub/ScanNet/utilities/dataset_utils.py:71: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  list_labels = np.array(list_labels)
/Users/jerometubiana/Documents/GitHub/ScanNet/utilities/dataset_utils.py:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  list_resids = np.array(list_resids)


## Instantiate sphere geometry
This cell must be executed first

In [3]:
sg = weight_logo_3d.make_sphere_geometry(30)

# Interactive visualization of one amino acid filter

In [ ]:
renderer = show_3d_filters.plot_aminoacid_filter(filter_specificities,89,sg=sg);
display(renderer)

# Interactive visualization of one atomic filter

In [27]:
renderer = show_3d_filters.plot_atomic_filter(filter_specificities,75,sg=sg,threshold1=0.33);
display(renderer)

Renderer(camera=PerspectiveCamera(position=(8.0, 5.0, 8.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.…

<Figure size 288x576 with 0 Axes>

<Figure size 288x576 with 0 Axes>

<Figure size 288x576 with 0 Axes>

<Figure size 288x576 with 0 Axes>

<Figure size 288x576 with 0 Axes>

<Figure size 288x576 with 0 Axes>

<Figure size 288x576 with 0 Axes>

<Figure size 288x576 with 0 Axes>

# Filter visualization with custom camera position

In [28]:
renderer = show_3d_filters.plot_atomic_filter(filter_specificities,30,sg=sg,
                                             camera_position=[-0.3, 0.6, 1.0]);

display(renderer)


Renderer(camera=PerspectiveCamera(position=(-3.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, …

<Figure size 288x576 with 0 Axes>

<Figure size 288x576 with 0 Axes>

<Figure size 288x576 with 0 Axes>

# Filter visualization with custom camera position, take screenshot

In [29]:
renderer = show_3d_filters.plot_atomic_filter(filter_specificities,30,sg=sg,
                                             camera_position=[-0.3, 0.6, 1.0]);
weight_logo_3d.make_screenshot(renderer,'screenshot_filter.png')
display(renderer)



Renderer(camera=PerspectiveCamera(position=(-3.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, …

<Figure size 288x576 with 0 Axes>

<Figure size 288x576 with 0 Axes>

<Figure size 288x576 with 0 Axes>